<a href="https://colab.research.google.com/github/AnsariZuhaib/1st-demo-project/blob/main/bank_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:



import sqlite3
import bcrypt

class BankDatabaseManager:
    def __init__(self, db="bank_database.db"):
        self.conn = sqlite3.connect(db)
        self.cursor = self.conn.cursor()
        self.create_users_table()
        self.create_transaction_table()

    # USERS TABLE
    def create_users_table(self):
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS users(
            serial_no INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            password_id TEXT UNIQUE,
            balance INTEGER DEFAULT 0,
            password TEXT,
            identification_number INTEGER
        );
        """)
        self.conn.commit()

    # TRANSACTION TABLE
    def create_transaction_table(self):
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS transactions(
            name TEXT,
            password_id TEXT,
            amount INTEGER,
            transaction_type TEXT,
            FOREIGN KEY(password_id) REFERENCES users(password_id)
        );
        """)
        self.conn.commit()

    def login(self):
        id_ = input("enter identification_number: ")
        password = input("enter password: ")
        self.cursor.execute("SELECT password FROM users WHERE identification_number = ?", (id_,))
        data = self.cursor.fetchone()
        if data:
          stored_password = data[0]
        if bcrypt.checkpw(password.encode('utf-8'), stored_password):
             print("correct password")
             return True
        else:
            print("incorrect password")
            return False

        if self.login():
              main.menu()
        else:
              print("login failed")

    def check_balance(self):
        name = input("Enter name: ")
        self.cursor.execute("SELECT balance FROM users WHERE name=?", (name,))
        data = self.cursor.fetchone()

        if data:
            print("Balance:", data[0])
        else:
            print("User not found")

    def credit(self):
        name = input("Enter name: ")
        amount = int(input("Amount: "))

        self.cursor.execute(
            "UPDATE users SET balance = balance + ? WHERE name=?",
            (amount, name)
        )

        self.cursor.execute("SELECT password_id FROM users WHERE name=?", (name,))
        password_id = self.cursor.fetchone()[0]

        self.cursor.execute(
            "INSERT INTO transactions VALUES(?,?,?,?)",
            (name, password_id, amount, "credit")
        )

        self.conn.commit()
        print("Amount credited")

    def debit(self):
        name = input("Enter name: ")
        amount = int(input("Amount: "))

        self.cursor.execute("SELECT balance FROM users WHERE name=?", (name,))
        data = self.cursor.fetchone()

        if not data or amount > data[0]:
            print("Insufficient balance")
            return

        self.cursor.execute(
            "UPDATE users SET balance = balance - ? WHERE name=?",
            (amount, name)
        )

        self.cursor.execute("SELECT password_id FROM users WHERE name=?", (name,))
        password_id = self.cursor.fetchone()[0]

        self.cursor.execute(
            "INSERT INTO transactions VALUES(?,?,?,?)",
            (name, password_id, amount, "debit")
        )

        self.conn.commit()
        print("Amount debited")

    def transfer(self):
        sender = input("Sender: ")
        receiver = input("Receiver: ")
        amount = int(input("Amount: "))

        self.cursor.execute("SELECT balance FROM users WHERE name=?", (sender,))
        data = self.cursor.fetchone()

        if not data or data[0] < amount:
            print("Insufficient balance")
            return

        # Update balances
        self.cursor.execute(
            "UPDATE users SET balance = balance - ? WHERE name=?",
            (amount, sender)
        )
        self.cursor.execute(
            "UPDATE users SET balance = balance + ? WHERE name=?",
            (amount, receiver)
        )

        # Get IDs
        self.cursor.execute("SELECT password_id FROM users WHERE name=?", (sender,))
        sender_id = self.cursor.fetchone()[0]

        self.cursor.execute("SELECT password_id FROM users WHERE name=?", (receiver,))
        receiver_id = self.cursor.fetchone()[0]

        # Log transactions
        self.cursor.execute(
            "INSERT INTO transactions VALUES(?,?,?,?)",
            (sender, sender_id, amount, "transfer")
        )
        self.cursor.execute(
            "INSERT INTO transactions VALUES(?,?,?,?)",
            (receiver, receiver_id, amount, "received")
        )

        self.conn.commit()
        print("Transfer successful")

    def transaction_history(self):
        password_id = input("Enter password id: ")

        self.cursor.execute("""
        SELECT users.name, transactions.amount, transactions.transaction_type
        FROM users
        INNER JOIN transactions
        ON users.password_id = transactions.password_id
        WHERE users.password_id = ?
        """, (password_id,))

        for row in self.cursor.fetchall():
            print(row)


    def menu(self):
        while True:
            print("\n1 Check Balance")
            print("2 Credit")
            print("3 Debit")
            print("4 Transfer")
            print("5 Transaction History")
            print("6 Exit")

            choice = input("Choice: ")

            if choice == "1":
                self.check_balance()
            elif choice == "2":
                self.credit()
            elif choice == "3":
                self.debit()
            elif choice == "4":
                self.transfer()
            elif choice == "5":
                self.transaction_history()
            elif choice == "6":
                break

        self.conn.close()

    def role_column(self):
      self.cursor.execute(
        "ALTER TABLE users ADD COLUMN role TEXT DEFAULT 'user'"
        )
      self.cursor.execute(
          "ALTER TABLE users ADD COLUMN is_active BOOLEAN DEFAULT TRUE"
      )
      self.conn.commit()

    def role_assign(self,id):
       self.cursor.execute(
            "UPDATE users SET role = 'admin' WHERE serial_no = ?", (id,)
        )
       self.conn.commit()

    def role_find(self):
        user_id = input("enter user id:")
        self.cursor.execute(
            "SELECT role FROM users WHERE id = ?,(user_id,)"
            )
        data = self.cursor.fetchone()
        if data and data[0] == 'admin':
           self.admin_menu()
        else:
           self.menu()

    def admin_menu(self):
      while True:
        print("/n select 1 to view data: ")
        print("select 2 to freeze account: ")
        print("select 3 to delete user: ")
        print("select 4 to unfreeze account: ")
        print("select 5 to view total balance in bank: ")
        print("select 6 to exit")
        admin_choice = int(input("enter your choice: "))

        if admin_choice == 1:
         self.cursor.execute(
            "SELECT name, id , is_active, balance FROM users"
        )
         data = self.cursor.fetchall()
         self.conn.commit()

        elif admin_choice == 2:
         user_id = input("enter user id : ")
         self.cursor.execute(
            "UPDATE users SET is_active = 0 WHERE id = ?,(user_id,)"
        )
         self.conn.commit()

        elif admin_choice == 3:
         user_id = (input("enter user id: "))
         self.cursor.execute(
            "DELETE FROM users WHERE id = ?,(user_id,)"
        )
         self.conn.commit()

        elif admin_choice == 4:
          user_id = input("enter user id: ")
          self.cursor.execute(
             "UPDATE users SET is_active = 1 WHERE id = ?,(user_id,)"
         )
          self.conn.commit()

        elif admin_choice == 5:
          self.cursor.execute(
             "SELECT SUM(balance) FROM users"
         )
          data = self.cursor.fetchone()
          self.conn.commit()

        elif admin_choice == 6:
          break


bank = BankDatabaseManager()
if bank.login():
    bank.menu()
else:
    print("Login failed")
    self.cursor.close()

ModuleNotFoundError: No module named 'bcrypt'

In [ ]:
pip install bcrypt